# EDA: Anonymized Companies
## Brooke Stevens

In [1165]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cleaning Companies

In [1166]:
companies_df = pd.read_csv("anonymized_hubspot_companies.csv")
companies_df.head()

,Parent Company,CCaaS,Annual Revenue,Target Account,Associated Contact,Number of Form Submissions,Total Agents,Web Technologies,Close Date,# of Agents Total,...,BPO Program,Primary Sub-Industry,Number of Sessions,WFM,Country/Region,Industry,Create Date,Company name,Last Modified Date,BPO Program Tier
0,NaN,NaN,1.000000e+09,NaN,Contact_ef780380,0.0,NaN,Route 53,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,HVAC and plumbing supply,2024-10-30 10:51,Company_4fc73a2a,2025-02-14 13:39,NaN
1,NaN,NaN,NaN,NaN,Contact_93373ba5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Pest Control,2024-09-12 18:30,Company_f3f7e884,2025-02-14 13:39,NaN
2,NaN,NaN,5.000000e+07,NaN,Contact_2e8e0993,0.0,NaN,Youtube; App Nexus; Google Tag Manager; Facebo...,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,Pest Control,2024-09-03 10:44,Company_89929bed,2025-02-14 13:39,NaN
3,NaN,Five9,5.000000e+07,NaN,Contact_635e44ed,0.0,NaN,Postmark; Facebook Advertiser; Salesforce; Goo...,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,Pest Control,2024-08-02 11:36,Company_33dbf591,2025-02-14 13:39,NaN
4,NaN,Five9,1.000000e+08,NaN,Contact_8d055096,0.0,NaN,Microsoft Office 365; Google Tag Manager; Reca...,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,Pest Control,2024-08-02 11:36,Company_a3079821,2025-02-14 13:39,NaN


Which columns should be included in the companies dataset, and how will missing values be handled?
- Parent Company: NO
- CCaaS: YES, add one-hot encoding
- Annual Revenue: YES and remove companies where this info is not available, turn this into binned categorical feature called Revenue Category (1-8 scale)
- Target Account: YES, include 1 for True and 0 for False/empty
- Associated Contact: NO
- Number of Form Submissions: YES, but change to 1 for >0 and 0 for =0 because companies with deals closed had a lot of form submissions, new feature is Form Submission YN
- Total Agents: NO
- Web Technologies: Remove companies where this info is not available, add one-hot encoding
- Close Date: Turn into binary 0 and 1 values, new feature is Close YN (target)
- \# of Agents Total: NO, data not available for negative class
- Number of times contacted: YES and remove companies where this info is not available
- Contact with Primary Company: NO
- ICP Fit Level: YES, Include 1 for ideal fit and 0 for not ideal fit
- Revenue range: NO
- \# of Agents Contracted: NO
- Record ID: YES
- Time Zone: NO
- Primary Industry: YES, add one-hot encoding
- Number of Pageviews: YES, include 0 for empty
- Primary Company: NO
- Year Founded: NO
- ICP: NO, this is redundant with ICP Fit Level
- Industry group: NO
- Segmentation: NO
- LMS System: NO
- SymTrain Use Cases: NO
- Associated Company: NO
- BPO: YES, add "None" for empty values, turn into one-hot encoding
- SSO Application: NO
- State/Region: NO
- SymTrain Product: NO
- Contract End Date: NO
- SSO Implemented? NO
- Consolodated Industry: YES, set (No value) to null, one-hot encoded
- Type: YES, add one-hot encoding
- Number of Employees: YES and remove companies where this info is not available, turn this into binned categorical feature called Company Size Category (1-8 scale)
- BPO Program: NO, redundant with other fields
- Primary Sub-Industry: NO, redundant with Industry
- Number of Sessions: YES, include 0 for empty
- WFM: YES, add one-hot encoding
- Country/Region: YES, remove companies with no information, one-hot encoding
- Industry: YES, remove companies with no information, one-hot encoding
- Create Date: Extract year, month, and quarter as INTs, delete this column though
- Company name: NO
- Last Modified Date: NO
- BPO Program Tier: NO

First, we will deal with the Web Technologies column

In [1167]:
companies_df = companies_df.dropna(subset=['Web Technologies'])

In [1168]:
import warnings
warnings.filterwarnings("ignore")

all_tools = set()

companies_df['Web Technologies'] = companies_df['Web Technologies'].apply(lambda x: x.split('; '))

for tools in companies_df['Web Technologies']:
    all_tools.update(tools)

for tool in all_tools:
    companies_df[f"Web Technologies_{tool}"] = companies_df['Web Technologies'].apply(lambda x: 1 if tool in x else 0)

companies_df = companies_df.drop(columns=['Web Technologies'])

companies_df.head()

,Parent Company,CCaaS,Annual Revenue,Target Account,Associated Contact,Number of Form Submissions,Total Agents,Close Date,# of Agents Total,Number of times contacted,...,Web Technologies_Fullcontact,Web Technologies_Taboola Ads,Web Technologies_Incapsula,Web Technologies_6sense,Web Technologies_Totango,Web Technologies_Oracle CRM,Web Technologies_Icims,Web Technologies_Oracle Hyperion,Web Technologies_ElasticEmail,Web Technologies_Invoca
0,NaN,NaN,1.000000e+09,NaN,Contact_ef780380,0.0,NaN,NaN,NaN,5.0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,5.000000e+07,NaN,Contact_2e8e0993,0.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Five9,5.000000e+07,NaN,Contact_635e44ed,0.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Five9,1.000000e+08,NaN,Contact_8d055096,0.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
6,NaN,Five9,1.000000e+08,NaN,Contact_d5af3473,0.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


Now, I will delete all the columns which I've decided are not helpful to the model prediction

In [1169]:
companies_df = companies_df.drop(['Parent Company',
              'Associated Contact',
              'Total Agents',
              '# of Agents Total',
              'Contact with Primary Company',
              'Revenue range',
              '# of Agents Contracted',
              'Time Zone',
              'Primary Company',
              'Year Founded',
              'ICP',
              'Industry group',
              'Segmentation',
              'LMS System',
              'SymTrain Use Cases',
              'Associated Company',
              'SSO Application',
              'State/Region',
              'SymTrain Product',
              'Contract End Date',
              'SSO Implemented?',
              'BPO Program',
              'Primary Sub-Industry',
              'Company name',
              'Last Modified Date',
              'BPO Program Tier'], axis=1)

CCaaS

In [1170]:
ccaas_encoded = pd.get_dummies(companies_df['CCaaS'], prefix='CCaaS', dtype='int')
companies_df = companies_df.drop(columns=['CCaaS']).join(ccaas_encoded)

Annual Revenue

In [1171]:
companies_df = companies_df.dropna(subset=['Annual Revenue'])
bins = [0, 1_000_000, 10_000_000, 50_000_000, 100_000_000, 500_000_000, 1_000_000_000, 10_000_000_000, float('inf')]  
# Ranges: <1M, 1M-10M, 10M-50M, 50M-100M, 100M-500M, 500M-1B, 1B-10B, 10B+
labels = [1, 2, 3, 4, 5, 6, 7, 8]

companies_df['Revenue Category'] = pd.cut(companies_df['Annual Revenue'], bins=bins, labels=labels, right=False)
companies_df = companies_df.drop(columns=['Annual Revenue'])

Target Account

In [1172]:
companies_df['Target Account'] = companies_df['Target Account'].fillna(False).astype(int)

Number of Form Submissions

In [1173]:
companies_df['Number of Form Submissions'] = companies_df['Number of Form Submissions'].fillna(0).astype(int)
companies_df['Form Submission YN'] = companies_df['Number of Form Submissions'].apply(lambda x: 1 if x > 0 else 0)
companies_df = companies_df.drop(columns=['Number of Form Submissions'])

Number of times contacted

In [1174]:
companies_df = companies_df.dropna(subset=['Number of times contacted'])
companies_df['Number of times contacted'] = companies_df['Number of times contacted'].astype(int)

ICP Fit Level

In [1175]:
companies_df['ICP Fit Level'] = np.where(companies_df['ICP Fit Level'] == 'Tier 1: Ideal Fit', 1, 0)

Primary Industry

In [1176]:
primary_industry_encoded = pd.get_dummies(companies_df['Primary Industry'], prefix='Primary Industry', dtype='int')
companies_df = companies_df.drop(columns=['Primary Industry']).join(primary_industry_encoded)

Number of Pageviews

In [1177]:
companies_df['Number of Pageviews'] = companies_df['Number of Pageviews'].fillna(0).astype(int)

BPO

In [1178]:
all_bpos = set()

companies_df['BPO'] = companies_df['BPO'].apply(lambda x: x.split('; ') if isinstance(x, str) else [])

for bpo in companies_df['BPO']:
    all_bpos.update(bpo)

for bpo in all_bpos:
    companies_df[f"BPO_{bpo}"] = companies_df['BPO'].apply(lambda x: 1 if isinstance(x, list) and bpo in x else 0)

companies_df = companies_df.drop(columns=['BPO'])

Consolidated Industry

In [1179]:
companies_df['Consolidated Industry'] = companies_df['Consolidated Industry'].replace("(No value)", np.nan)
consolidated_industry_encoded = pd.get_dummies(companies_df['Consolidated Industry'], prefix='Consolidated Industry', dtype='int')
companies_df = companies_df.drop(columns=['Consolidated Industry']).join(consolidated_industry_encoded)

Type

In [1180]:
type_encoded = pd.get_dummies(companies_df['Type'], prefix='Type', dtype='int')
companies_df = companies_df.drop(columns=['Type']).join(type_encoded)

Number of Employees

In [1181]:
companies_df = companies_df.dropna(subset=['Number of Employees'])
companies_df['Number of Employees'] = companies_df['Number of Employees'].astype(int)

bins = [0, 10, 50, 200, 500, 1000, 5000, 10000, float('inf')]
# Ranges: 0-10, 11-50, 51-200, 201-500, 501-1000, 1001-5000, 5001-10000, 10001+
labels = [1, 2, 3, 4, 5, 6, 7, 8]

companies_df['Company Size Category'] = pd.cut(companies_df['Number of Employees'], bins=bins, labels=labels, right=True)
companies_df = companies_df.drop(columns=['Number of Employees'])

Number of Sessions

In [1182]:
companies_df['Number of Sessions'] = companies_df['Number of Sessions'].fillna(0)
companies_df['Number of Sessions'] = companies_df['Number of Sessions'].astype(int)

WFM

In [1183]:
wfm_encoded = pd.get_dummies(companies_df['WFM'], prefix='WFM', dtype='int')
companies_df = companies_df.drop(columns=['WFM']).join(wfm_encoded)
companies_df.head()

,Target Account,Close Date,Number of times contacted,ICP Fit Level,Record ID,Number of Pageviews,Number of Sessions,Country/Region,Industry,Create Date,...,Type_Former Customer,Type_In Trial,Type_Integrator,Type_Other,Type_Partner,Type_Prospect,Type_Suspect,Type_Vendor,Company Size Category,WFM_Verint
0,0,NaN,5,0,25246439235,0,0,United States,HVAC and plumbing supply,2024-10-30 10:51,...,0,0,0,0,0,0,0,0,6,0
7,0,NaN,0,0,25469822048,0,0,United States,Pest Control,2024-06-27 05:21,...,0,0,0,0,0,0,0,0,7,0
9,0,NaN,7,0,25474760021,0,0,United States,Pet Care,2024-06-13 08:14,...,0,0,0,0,0,0,0,0,5,0
10,0,NaN,27,1,13512014593,0,0,United States,Pet Care,2023-02-15 07:55,...,0,0,0,0,0,0,0,0,8,0
11,0,NaN,2,0,17733315247,0,0,Denmark,Jewelry,2023-10-19 12:19,...,0,0,0,0,0,0,0,0,8,0


Country/Region

In [1184]:
companies_df = companies_df.dropna(subset=['Country/Region'])
country_region_encoded = pd.get_dummies(companies_df['Country/Region'], prefix='Country/Region', dtype='int', drop_first=True)
companies_df = companies_df.drop(columns=['Country/Region']).join(country_region_encoded)

Industry

In [1185]:
industry_encoded = pd.get_dummies(companies_df['Industry'], prefix='Industry', dtype='int')
companies_df = companies_df.drop(columns=['Industry']).join(industry_encoded)
companies_df.head()

,Target Account,Close Date,Number of times contacted,ICP Fit Level,Record ID,Number of Pageviews,Number of Sessions,Create Date,Web Technologies_Typeform,Web Technologies_SOASTA,...,Industry_Transportation,Industry_Transportation/Trucking/Railroad,Industry_Utilities,Industry_Venture Capital & Private Equity,Industry_Veterinary,Industry_Warehousing,Industry_Wholesale,Industry_Wine and Spirits,Industry_Wireless,Industry_Writing and Editing
0,0,NaN,5,0,25246439235,0,0,2024-10-30 10:51,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,NaN,0,0,25469822048,0,0,2024-06-27 05:21,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,NaN,7,0,25474760021,0,0,2024-06-13 08:14,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,NaN,27,1,13512014593,0,0,2023-02-15 07:55,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,NaN,2,0,17733315247,0,0,2023-10-19 12:19,0,0,...,0,0,0,0,0,0,0,0,0,0


Create Date

In [1186]:
companies_df['Create Date'] = pd.to_datetime(companies_df['Create Date'])

companies_df['Create Date_Year'] = companies_df['Create Date'].dt.year
companies_df['Create Date_Month'] = companies_df['Create Date'].dt.month
companies_df['Create Date_Quarter'] = companies_df['Create Date'].dt.quarter
companies_df = companies_df.drop(columns=['Create Date'])

Close YN (target)

In [1187]:
companies_df['Close YN'] = np.where(companies_df['Close Date'].isna(), 0, 1)
companies_df = companies_df.drop(columns=['Close Date'])

In [1188]:
companies_df.head()

,Target Account,Number of times contacted,ICP Fit Level,Record ID,Number of Pageviews,Number of Sessions,Web Technologies_Typeform,Web Technologies_SOASTA,Web Technologies_Gravatar,Web Technologies_Google Places,...,Industry_Veterinary,Industry_Warehousing,Industry_Wholesale,Industry_Wine and Spirits,Industry_Wireless,Industry_Writing and Editing,Create Date_Year,Create Date_Month,Create Date_Quarter,Close YN
0,0,5,0,25246439235,0,0,0,0,0,0,...,0,0,0,0,0,0,2024,10,4,0
7,0,0,0,25469822048,0,0,1,0,0,0,...,0,0,0,0,0,0,2024,6,2,0
9,0,7,0,25474760021,0,0,0,0,0,0,...,0,0,0,0,0,0,2024,6,2,0
10,0,27,1,13512014593,0,0,0,0,0,0,...,0,0,0,0,0,0,2023,2,1,0
11,0,2,0,17733315247,0,0,0,0,0,0,...,0,0,0,0,0,0,2023,10,4,0


Notes:
- The Record ID column will help map the datasets together, this feature should be removed when training the model
- The target variable predicting whether a deal is closed is called ‘Close YN’ and it is a 0/1 binary categorical variable

# Cleaning Tickets

In [1189]:
tickets_df = pd.read_csv("anonymized_hubspot_tickets.csv")
tickets_df.head()

,Create date,Ticket Tags,Stage Date - Project Launch,Associated Contact,Category,Target Launch Date,Kickoff Call,Close date,Pipeline,Stage Date - Project Initiation,...,Latest Milestone Update Date,Ticket name,Trial Overview,Trial Start Date,1st Syms presented for review,Project Launch Day,Training: Reporting,Construction of 1st Sym begun,Trial Required,Was the sym QAed?
0,12/18/23 15:02,NaN,NaN,Contact_b6d930b5,NaN,NaN,12/27/23,1/29/24 11:50,Project Success Plan,12/18/23,...,NaN,Ticket_4baf7886,NaN,NaN,NaN,NaN,NaN,12/27/23,NaN,NaN
1,10/30/24 12:31,NaN,NaN,Contact_aa84b2e1,NaN,NaN,NaN,2/10/25 11:18,Project Success Plan,NaN,...,NaN,Ticket_88dda9e1,NaN,11/5/24,NaN,NaN,NaN,NaN,NaN,NaN
2,12/12/24 9:01,NaN,1/10/25,Contact_fec7b624,NaN,NaN,12/10/24,NaN,Project Success Plan,12/13/24,...,NaN,Ticket_2226dda1,Trial until end of April,12/11/24,12/23/24,1/6/25,12/23/24,12/11/24,NaN,NaN
3,9/7/23 16:37,NaN,NaN,Contact_11c92d4d,NaN,10/2/23,6/8/23,1/4/24 13:28,Project Success Plan,NaN,...,11/24/23,Ticket_6c996e04,"Coaching already implemented, adding onboardin...",NaN,NaN,6/14/23,7/19/23,NaN,NaN,NaN
4,2/14/25 13:34,NaN,NaN,Contact_78b60b6b,NaN,NaN,NaN,NaN,Project Success Plan,2/14/25,...,NaN,Ticket_14c6e9a9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Which columns should be included in the tickets dataset, and how will missing values be handled? (This dataset is hard to handle because we only have 79 examples! I will prioritize keeping columns full while eliminating other columns that have a fair amount of missingness)
- Create date: Get other data from this value, but then delete
- Ticket Tags: NO
- Stage Date - Project Launch: NO
- Associated Contact: NO
- Category: NO
- Target Launch Date: NO
- Kickoff Call: NO
- Close date: Delete missing values, get other data from this value, but then delete
- Pipeline: NO
- Stage Date - Project Initiation: NO
- Stage Date - Planning Phase: NO
- Ticket status: YES, add one-hot encoding, numbers are assigned to each stage and a one-hot encoding is assigned for a win or loss (Ticket status_Won and Ticket status_Lost)
- Stage Date - Execution: NO
- Time to first agent email reply (HH:mm:ss): NO, this is duplicate data from Response time (HH:mm:ss)
- Last CES survey rating: NO
- Priority: NO
- Associated Company: NO
- Who will be creating the Syms?: NO
- Stage Date - Closure Phase: NO
- Stage Date - Monitoring and Control Phase: NO
- Associated Company (Primary): NO
- Response time (HH:mm:ss): YES, convert to hours as Response time hours
- Ticket ID: YES
- Time to close (HH:mm:ss): NO, data only present for closed tickets
- Associated Deal: NO
- Library index approved: NO
- Training: General Overview: Get other data from this value (count if date is present, don't if date is not), but then delete
- Latest Milestone: NO
- Training: Deployment/User Management Training: Get other data from this value (count if date is present, don't if date is not), but then delete
- Requirements for the Trial: YES, one-hot encoding
- Training: Sym Building 101: Get other data from this value (count if date is present, don't if date is not), but then delete
- Last modified date: NO
- Type: NO
- Trial End Date: NO
- Training: Sym Building 201: Get other data from this value (count if date is present, don't if date is not), but then delete
- Stage Date - Converted Won: NO
- Latest Milestone Update Date: NO
- Ticket name: NO
- Trial Overview: NO
- Trial Start Date: NO
- 1st Syms presented for review: NO
- Project Launch Day: NO
- Training: Reporting: Get other data from this value (count if date is present, don't if date is not), but then delete
- Construction of 1st Sym begun: NO
- Trial Required: NO
- Was the sym QAed?: NO

Now, I will delete all the columns which I've decided are not helpful to the model prediction

In [1190]:
tickets_df = tickets_df.drop(['Ticket Tags',
                              'Stage Date - Project Launch',
                              'Associated Contact',
                              'Category',
                              'Target Launch Date',
                              'Kickoff Call',
                              'Pipeline',
                              'Stage Date - Project Initiation',
                              'Stage Date - Planning Phase',
                              'Stage Date - Execution',
                              'Time to first agent email reply (HH:mm:ss)',
                              'Last CES survey rating',
                              'Priority',
                              'Associated Company',
                              'Who will be creating the Syms?',
                              'Stage Date - Closure Phase',
                              'Stage Date - Monitoring and Control Phase',
                              'Associated Company (Primary)',
                              'Associated Deal',
                              'Library index approved',
                              'Latest Milestone',
                              'Last modified date',
                              'Type',
                              'Stage Date - Converted Won',
                              'Latest Milestone Update Date',
                              'Ticket name',
                              'Trial Overview',
                              '1st Syms presented for review',
                              'Project Launch Day',
                              'Construction of 1st Sym begun',
                              'Trial Required',
                              'Was the sym QAed?',
                              'Trial Start Date',
                              'Trial End Date',
                              'Time to close (HH:mm:ss)'], axis=1)

In [1191]:
tickets_df.head()

,Create date,Close date,Ticket status,Response time (HH:mm:ss),Ticket ID,Training: General Overview,Training: Deployment/User Management Training,Requirements for the Trial,Training: Sym Building 101,Training: Sym Building 201,Training: Reporting
0,12/18/23 15:02,1/29/24 11:50,Converted-Won,NaN,2193923207,NaN,NaN,Coaching,NaN,NaN,NaN
1,10/30/24 12:31,2/10/25 11:18,Lost,49:24:00,15899166073,NaN,NaN,NaN,NaN,NaN,NaN
2,12/12/24 9:01,NaN,Project Launch,1516:30:41,17452741013,12/23/24,12/23/24,Assessment; Coaching,12/23/24,12/23/24,12/23/24
3,9/7/23 16:37,1/4/24 13:28,Converted-Won,26:25:51,1891106581,7/12/23,7/19/23,Onboarding,7/11/23,8/1/23,7/19/23
4,2/14/25 13:34,NaN,Planning Phase,13:32:11,20395039397,NaN,NaN,NaN,NaN,NaN,NaN


Create date

In [1192]:
tickets_df['Create date'] = pd.to_datetime(tickets_df['Create date'])

tickets_df['Create date_Year'] = tickets_df['Create date'].dt.year
tickets_df['Create date_Month'] = tickets_df['Create date'].dt.month
tickets_df['Create date_Quarter'] = tickets_df['Create date'].dt.quarter

Close date/Implementation Duration Days

In [1193]:
tickets_df = tickets_df.dropna(subset=['Close date'])
tickets_df['Close date'] = pd.to_datetime(tickets_df['Close date'])
tickets_df['Implementation Duration Days'] = (tickets_df['Close date'] - tickets_df['Create date']).dt.days

tickets_df = tickets_df.drop(columns=['Create date', 'Close date'])

Ticket status

| Ticket Status               | Numeric Order |
|-----------------------------|---------------|
| Project Initiation Phase     | 1             |
| Planning Phase               | 2             |
| Project Launch               | 3             |
| Execution Phase              | 4             |
| Monitoring and Control Phase | 5             |
| Closure Phase                | 6             |
| Converted-Won                | 7 (Success)   |
| Lost                         | 7 (Failure)   |

In [1194]:
status_order = {
    "Project Initiation Phase": 1,
    "Planning Phase": 2,
    "Project Launch": 3,
    "Execution Phase": 4,
    "Monitoring and Control Phase": 5,
    "Closure Phase": 6,
    "Converted-Won": 7,
    "Lost": 7
}

tickets_df["Status Order"] = tickets_df["Ticket status"].map(status_order)

tickets_df["Ticket status_Won"] = tickets_df["Ticket status"].apply(lambda x: 1 if x == "Converted-Won" else 0)
tickets_df["Ticket status_Lost"] = tickets_df["Ticket status"].apply(lambda x: 1 if x == "Lost" else 0)

tickets_df["Ticket status"] = tickets_df["Status Order"]
tickets_df = tickets_df.drop(columns=['Status Order'])

Response time

In [1195]:
tickets_df = tickets_df.dropna(subset=['Response time (HH:mm:ss)'])

tickets_df['Response time (HH:mm:ss)'] = pd.to_timedelta(tickets_df['Response time (HH:mm:ss)'])

tickets_df["Response time hours"] = tickets_df['Response time (HH:mm:ss)'].dt.total_seconds() / 3600

tickets_df = tickets_df.drop(columns=['Response time (HH:mm:ss)'])

All training columns:

In [1196]:
training_columns = ['Training: General Overview',
                    'Training: Deployment/User Management Training',
                    'Training: Sym Building 101',
                    'Training: Sym Building 201',
                    'Training: Reporting']

tickets_df["Training Completion Count"] = tickets_df[training_columns].notna().sum(axis=1)

tickets_df = tickets_df.drop(columns=training_columns)

Requirements for the Trial

In [1197]:
all_reqs = set()

tickets_df['Requirements for the Trial'] = tickets_df['Requirements for the Trial'].apply(lambda x: x.split('; ') if isinstance(x, str) else [])

for req in tickets_df['Requirements for the Trial']:
    all_reqs.update(req)

for req in all_reqs:
    tickets_df[f"Requirements for the Trial_{req}"] = tickets_df['Requirements for the Trial'].apply(lambda x: 1 if isinstance(x, list) and req in x else 0)

tickets_df = tickets_df.drop(columns=['Requirements for the Trial'])

In [1198]:
tickets_df.head()

,Ticket status,Ticket ID,Create date_Year,Create date_Month,Create date_Quarter,Implementation Duration Days,Ticket status_Won,Ticket status_Lost,Response time hours,Training Completion Count,Requirements for the Trial_Onboarding,Requirements for the Trial_Assessment,Requirements for the Trial_Coaching
1,7,15899166073,2024,10,4,102,0,1,49.400000,0,0,0,0
3,7,1891106581,2023,9,3,118,1,0,26.430833,5,1,0,0
5,7,3049197180,2024,7,3,142,0,1,0.100000,5,1,0,0
6,7,3049195860,2024,7,3,142,0,1,0.083056,5,1,0,0
7,7,3295514681,2024,9,3,101,1,0,156.109722,0,1,0,0


Notes:
- The Ticket ID column will help map the datasets together, this feature should be removed when training the model
- The target variable for predicting whether a ticket is closed and won is called ‘Ticket status_Won’ and it is a 0/1 binary categorical variable
    - Equaling 0 does not necessarily mean it's a loss, the ticket still could be in process

# Cleaning Deals

In [1199]:
deals_df = pd.read_csv("anonymized_hubspot_deals.csv")
deals_df.head()

,Weighted amount,Deal Description,"Cumulative time in ""BANT Deal. Pain ID'ed (Sales Pipeline)"" (HH:mm:ss)","Cumulative time in ""Opportunity (Sales Pipeline)"" (HH:mm:ss)",Days to close,Deal Score,Close Date,Deal source attribution 2,"Cumulative time in ""In Trial - Trial in Progress (Sales Pipeline)"" (HH:mm:ss)",Contract Start Date,...,"Cumulative time in ""Negotiation (Sales Pipeline)"" (HH:mm:ss)",Is Open (numeric),"Cumulative time in ""Renewals (Sales Pipeline)"" (HH:mm:ss)",Create Date,"Cumulative time in ""Contract Sent (Sales Pipeline)"" (HH:mm:ss)",Total contract value,Last Modified Date,"Cumulative time in ""Closed Won (Sales Pipeline)"" (HH:mm:ss)",Deal Stage,Deal Type
0,250.0,40-50 employees,NaN,NaN,69.0,33.0,2025-05-01 15:26,Event,NaN,NaN,...,NaN,1.0,NaN,2025-02-20 15:27,NaN,NaN,2025-02-20 16:28,NaN,Opportunity,New
1,0.0,"3,000 + Agents.",NaN,NaN,149.0,13.0,2025-07-20 09:06,Referral Partner,NaN,NaN,...,NaN,1.0,NaN,2025-02-20 08:07,NaN,NaN,2025-02-20 09:08,NaN,Partner Referrals,New
2,0.0,5-6k agents,NaN,NaN,149.0,13.0,2025-07-20 09:03,Referral Partner,NaN,NaN,...,NaN,1.0,NaN,2025-02-20 08:04,NaN,NaN,2025-02-20 09:05,NaN,Partner Referrals,New
3,0.0,Furthest Along - 300 agents,NaN,NaN,149.0,13.0,2025-07-20 08:55,Referral Partner,NaN,NaN,...,NaN,1.0,NaN,2025-02-20 07:58,NaN,NaN,2025-02-20 08:59,NaN,Partner Referrals,New
4,10000.0,BPO through partnership with AmplifAI,NaN,NaN,120.0,37.0,2025-06-20 15:01,Master Agent,NaN,NaN,...,NaN,1.0,NaN,2025-02-19 14:02,NaN,NaN,2025-02-19 15:03,NaN,Deep Dive. PSP Drafted,New


Which columns should be included in the deals dataset, and how will missing values be handled? (This dataset is hard to handle because we only have 79 examples! I will prioritize keeping columns full while eliminating other columns that have a fair amount of missingness)
- Weighted amount: YES
- Deal Description: NO
- Cumulative time in "BANT Deal. Pain ID'ed (Sales Pipeline)" (HH:mm:ss): NO
- Cumulative time in "Opportunity (Sales Pipeline)" (HH:mm:ss) NO
- Days to close: NO
- Deal Score: YES, categorical bins which is then one-hot encoded
- Close Date: NO, not all deals closed
- Deal source attribution 2: YES, add one-hot encoding
- Cumulative time in "In Trial - Trial in Progress (Sales Pipeline)" (HH:mm:ss): NO
- Contract Start Date: NO
- Cumulative time in "Partner Referrals  (Sales Pipeline)" (HH:mm:ss): NO
- Cumulative time in "Closed Lost (Sales Pipeline)" (HH:mm:ss): NO
- Cumulative time in "Deep Dive. PSP Drafted (Sales Pipeline)" (HH:mm:ss): NO
- Pipeline: NO
- Record ID: YES
- Forecast category: YES, add one-hot encoding
- Original Traffic Source: YES, add one-hot encoding
- Associated Company: NO
- Deal owner: NO
- Annual contract value: NO
- Is Closed (numeric): NO, we have deal stage
- Amount in company currency: NO, redundant of Amount
- Annual recurring revenue: NO
- Monthly recurring revenue: NO
- Deal probability: YES
- Associated Company (Primary): NO
- Is Closed Won: YES, convert to 0/1 binary
- Forecast probability: NO
- Contract End Date: NO
- Last Activity Date: NO
- Contract Term (Months): NO
- Trial Start date: NO
- Is closed lost: YES, convert to 0/1 binary
- Weighted amount in company currency: NO, redundant of Weighted amount
- Is Deal Closed?: NO
- Trial End Date: NO
- Deal Name: NO
- Amount: YES
- Cumulative time in "Closed Trial (Sales Pipeline)" (HH:mm:ss): NO
- Forecast amount: NO, redundant of amount
- Cumulative time in "Negotiation (Sales Pipeline)" (HH:mm:ss): NO
- Is Open (numeric): NO, we have deal stage and closed win/lost
- Cumulative time in "Renewals  (Sales Pipeline)" (HH:mm:ss): NO
- Create Date: Get other data from this value and then delete
- Cumulative time in "Contract Sent (Sales Pipeline)" (HH:mm:ss): NO
- Total contract value: NO
- Last Modified Date: NO
- Cumulative time in "Closed Won (Sales Pipeline)" (HH:mm:ss): NO
- Deal Stage: YES, categorical label encoding
- Deal Type: YES, one-hot encoding

Now, I will delete all the columns which I've decided are not helpful to the model prediction

In [1200]:
deals_df = deals_df.drop(["Deal Description",
                          "Cumulative time in \"BANT Deal. Pain ID'ed (Sales Pipeline)\" (HH:mm:ss)",
                          "Cumulative time in \"Opportunity (Sales Pipeline)\" (HH:mm:ss)",
                          "Days to close",
                          "Close Date",
                          "Cumulative time in \"In Trial - Trial in Progress (Sales Pipeline)\" (HH:mm:ss)",
                          "Contract Start Date",
                          "Cumulative time in \"Partner Referrals  (Sales Pipeline)\" (HH:mm:ss)",
                          "Cumulative time in \"Closed Lost (Sales Pipeline)\" (HH:mm:ss)",
                          "Cumulative time in \"Deep Dive. PSP Drafted (Sales Pipeline)\" (HH:mm:ss)",
                          "Pipeline",
                          "Associated Company",
                          "Deal owner",
                          "Annual contract value",
                          "Is Closed (numeric)",
                          "Amount in company currency",
                          "Annual recurring revenue",
                          "Monthly recurring revenue",
                          "Associated Company (Primary)",
                          "Forecast probability",
                          "Contract End Date",
                          "Last Activity Date",
                          "Contract Term (Months)",
                          "Trial Start date",
                          "Weighted amount in company currency",
                          "Is Deal Closed?",
                          "Trial End Date",
                          "Deal Name",
                          "Cumulative time in \"Closed Trial (Sales Pipeline)\" (HH:mm:ss)",
                          "Forecast amount",
                          "Cumulative time in \"Negotiation (Sales Pipeline)\" (HH:mm:ss)",
                          "Is Open (numeric)",
                          "Cumulative time in \"Renewals  (Sales Pipeline)\" (HH:mm:ss)",
                          "Cumulative time in \"Contract Sent (Sales Pipeline)\" (HH:mm:ss)",
                          "Total contract value",
                          "Last Modified Date",
                          "Cumulative time in \"Closed Won (Sales Pipeline)\" (HH:mm:ss)"], axis=1)

In [1201]:
deals_df.head()

,Weighted amount,Deal Score,Deal source attribution 2,Record ID,Forecast category,Original Traffic Source,Deal probability,Is Closed Won,Is closed lost,Amount,Create Date,Deal Stage,Deal Type
0,250.0,33.0,Event,33774471298,Pipeline,Offline Sources,0.20,False,False,1250.0,2025-02-20 15:27,Opportunity,New
1,0.0,13.0,Referral Partner,33735737768,Not forecasted,Offline Sources,0.05,False,False,0.0,2025-02-20 08:07,Partner Referrals,New
2,0.0,13.0,Referral Partner,33758455618,Not forecasted,Offline Sources,0.05,False,False,0.0,2025-02-20 08:04,Partner Referrals,New
3,0.0,13.0,Referral Partner,33741931267,Not forecasted,Offline Sources,0.05,False,False,0.0,2025-02-20 07:58,Partner Referrals,New
4,10000.0,37.0,Master Agent,33724679941,Best case,Offline Sources,0.20,False,False,50000.0,2025-02-19 14:02,Deep Dive. PSP Drafted,New


Deal Score

In [1202]:
bins = [0, 20, 40, 60, 80, 100]
# Ranges: 0-20, 21-40, 41-60, 61-80, 81-100
labels = [1, 2, 3, 4, 5]

deals_df['Deal Score'] = pd.cut(deals_df['Deal Score'], bins=bins, labels=labels, right=True)
dscore_encoded = pd.get_dummies(deals_df['Deal Score'], prefix='Deal Score', dtype='int')
deals_df = deals_df.drop(columns=['Deal Score']).join(dscore_encoded)

Deal source attribution 2

In [1203]:
dsa_2_encoded = pd.get_dummies(deals_df['Deal source attribution 2'], prefix='Deal source attribution 2', dtype='int')
deals_df = deals_df.drop(columns=['Deal source attribution 2']).join(dsa_2_encoded)

Forecast category

In [ ]:
forecast_cat_encoded = pd.get_dummies(deals_df['Forecast category'], prefix='Forecast category', dtype='int', drop_first=True)
deals_df = deals_df.drop(columns=['Forecast category']).join(forecast_cat_encoded)

,Weighted amount,Record ID,Original Traffic Source,Deal probability,Is Closed Won,Is closed lost,Amount,Create Date,Deal Stage,Deal Type,...,Deal source attribution 2_PLG,Deal source attribution 2_Referral Partner,Deal source attribution 2_Search,Deal source attribution 2_Social Media,Deal source attribution 2_SymTrain EE name,Deal source attribution 2_Verint Intro,Forecast category_Closed won,Forecast category_Commit,Forecast category_Not forecasted,Forecast category_Pipeline
0,250.0,33774471298,Offline Sources,0.20,False,False,1250.0,2025-02-20 15:27,Opportunity,New,...,0,0,0,0,0,0,0,0,0,1
1,0.0,33735737768,Offline Sources,0.05,False,False,0.0,2025-02-20 08:07,Partner Referrals,New,...,0,1,0,0,0,0,0,0,1,0
2,0.0,33758455618,Offline Sources,0.05,False,False,0.0,2025-02-20 08:04,Partner Referrals,New,...,0,1,0,0,0,0,0,0,1,0
3,0.0,33741931267,Offline Sources,0.05,False,False,0.0,2025-02-20 07:58,Partner Referrals,New,...,0,1,0,0,0,0,0,0,1,0
4,10000.0,33724679941,Offline Sources,0.20,False,False,50000.0,2025-02-19 14:02,Deep Dive. PSP Drafted,New,...,0,0,0,0,0,0,0,0,0,0


Original Traffic Source

In [1205]:
orig_traf_encoded = pd.get_dummies(deals_df['Original Traffic Source'], prefix='Original Traffic Source', dtype='int', drop_first=True)
deals_df = deals_df.drop(columns=['Original Traffic Source']).join(orig_traf_encoded)

Is Closed Won

In [1206]:
deals_df['Is Closed Won'] = deals_df['Is Closed Won'].astype(int)

Is closed lost

In [1207]:
deals_df['Is closed lost'] = deals_df['Is closed lost'].astype(int)

Create Date

In [1208]:
deals_df['Create Date'] = pd.to_datetime(deals_df['Create Date'])

deals_df['Create Date_Year'] = deals_df['Create Date'].dt.year
deals_df['Create Date_Month'] = deals_df['Create Date'].dt.month
deals_df['Create Date_Quarter'] = deals_df['Create Date'].dt.quarter

deals_df = deals_df.drop(columns=['Create Date'])

In [1209]:
deals_df.head()

,Weighted amount,Record ID,Deal probability,Is Closed Won,Is closed lost,Amount,Deal Stage,Deal Type,Deal Score_1,Deal Score_2,...,Forecast category_Not forecasted,Forecast category_Pipeline,Original Traffic Source_Offline Sources,Original Traffic Source_Organic Search,Original Traffic Source_Organic Social,Original Traffic Source_Other Campaigns,Original Traffic Source_Referrals,Create Date_Year,Create Date_Month,Create Date_Quarter
0,250.0,33774471298,0.20,0,0,1250.0,Opportunity,New,0,1,...,0,1,1,0,0,0,0,2025,2,1
1,0.0,33735737768,0.05,0,0,0.0,Partner Referrals,New,1,0,...,1,0,1,0,0,0,0,2025,2,1
2,0.0,33758455618,0.05,0,0,0.0,Partner Referrals,New,1,0,...,1,0,1,0,0,0,0,2025,2,1
3,0.0,33741931267,0.05,0,0,0.0,Partner Referrals,New,1,0,...,1,0,1,0,0,0,0,2025,2,1
4,10000.0,33724679941,0.20,0,0,50000.0,Deep Dive. PSP Drafted,New,0,1,...,0,0,1,0,0,0,0,2025,2,1


Deal Stage

| Deal Stage                    | Number |
|-------------------------------|--------|
| Partner Referrals              | 1      |
| Opportunity                    | 2      |
| BANT Deal. Pain ID'ed          | 3      |
| Negotiation                    | 4      |
| Deep Dive. PSP Drafted        | 5      |
| Contract Sent                  | 6      |
| Closed Won                     | 7      |
| Closed Lost                    | 7      |
| Closed Trial                   | 7      |
| Renewals                       | 8     |
| In Trial - Trial in Progress  | 9     |

In [ ]:
deal_stage_mapping = {
    "Partner Referrals ": 1,
    "Opportunity": 2,
    "BANT Deal. Pain ID'ed": 3,
    "Negotiation": 4,
    "Deep Dive. PSP Drafted": 5,
    "Contract Sent": 6,
    "Closed Won": 7,
    "Closed Lost": 7,
    "Closed Trial": 7,
    "Renewals ": 8,
    "In Trial - Trial in Progress": 9
}

deals_df["Deal Stage Categorical"] = deals_df["Deal Stage"].map(deal_stage_mapping)
deals_df["Deal Stage"] = deals_df["Deal Stage Categorical"].astype(int)
deals_df = deals_df.drop(columns=['Deal Stage Categorical'])

,Weighted amount,Record ID,Deal probability,Is Closed Won,Is closed lost,Amount,Deal Stage,Deal Type,Deal Score_1,Deal Score_2,...,Forecast category_Not forecasted,Forecast category_Pipeline,Original Traffic Source_Offline Sources,Original Traffic Source_Organic Search,Original Traffic Source_Organic Social,Original Traffic Source_Other Campaigns,Original Traffic Source_Referrals,Create Date_Year,Create Date_Month,Create Date_Quarter
0,250.0,33774471298,0.20,0,0,1250.0,2,New,0,1,...,0,1,1,0,0,0,0,2025,2,1
1,0.0,33735737768,0.05,0,0,0.0,1,New,1,0,...,1,0,1,0,0,0,0,2025,2,1
2,0.0,33758455618,0.05,0,0,0.0,1,New,1,0,...,1,0,1,0,0,0,0,2025,2,1
3,0.0,33741931267,0.05,0,0,0.0,1,New,1,0,...,1,0,1,0,0,0,0,2025,2,1
4,10000.0,33724679941,0.20,0,0,50000.0,5,New,0,1,...,0,0,1,0,0,0,0,2025,2,1


Deal Type

In [1212]:
deal_type_encoded = pd.get_dummies(deals_df['Deal Type'], prefix='Deal Type', dtype='int', drop_first=True)
deals_df = deals_df.drop(columns=['Deal Type']).join(deal_type_encoded)

In [1213]:
deals_df.head()

,Weighted amount,Record ID,Deal probability,Is Closed Won,Is closed lost,Amount,Deal Stage,Deal Score_1,Deal Score_2,Deal Score_3,...,Original Traffic Source_Organic Search,Original Traffic Source_Organic Social,Original Traffic Source_Other Campaigns,Original Traffic Source_Referrals,Create Date_Year,Create Date_Month,Create Date_Quarter,Deal Type_New,Deal Type_PS,Deal Type_Renewal
0,250.0,33774471298,0.20,0,0,1250.0,2,0,1,0,...,0,0,0,0,2025,2,1,1,0,0
1,0.0,33735737768,0.05,0,0,0.0,1,1,0,0,...,0,0,0,0,2025,2,1,1,0,0
2,0.0,33758455618,0.05,0,0,0.0,1,1,0,0,...,0,0,0,0,2025,2,1,1,0,0
3,0.0,33741931267,0.05,0,0,0.0,1,1,0,0,...,0,0,0,0,2025,2,1,1,0,0
4,10000.0,33724679941,0.20,0,0,50000.0,5,0,1,0,...,0,0,0,0,2025,2,1,1,0,0


Notes:
- The Record ID column will help map the datasets together, this feature should be removed when training the model
- The target variable for predicting whether a deal is closed and won is called ‘Is Closed Won’ and it is a 0/1 binary categorical variable
    - Equaling 0 does not necessarily mean it's a loss, the deal could be in trial